In [1]:
import api_files.ml_library as calc
import pandas as pd
from train_test_divide import extract_spammers_scrapers, extract_normal, split_train_test
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Hur kommer ett request se ut när det kommer in?
- sträng i Redis? 
- sparas i sorted sets?
- hämta som mest ett fixt antal, ex 50.

Efter allt är hämtat från Redis:
- spara undan i en data frame med varje värde i rätt kolumn.

Dataframe:

userID, sessionID, URL, osv.        time mellan req         unika sessionID
user1   session1, login
user1   session1  chat
user1   session1  sendMessage

Kör in data framen i en funktion som gör nedanstående steg med hjälp av diverse helper-metoder:
För user1: 
- vad är den genomsnittliga tiden mellan requests?
- tokens under 5 min (skapa en funktion som först grupperar och sen räknar ut medelvärde av antal sessionIDs för alla 5-min intervall)
När dessa två steg är klara, behöver vi inte längre ta hänsyn till timestamps eller sessionID. Resultaten från dessa beräkningar 
sparas undan i egna variabler som läggs till i slutskedet av konstruktionen av vår test-dataframe.

- vektorisera URLer och ta fram:
    - varians, 
    - unika requests,
    - längsta följd av upprepade förfrågningar

In [2]:
"""Code for fetching the data"""

#Fetch data from csv file
data = calc.read_csv_file('csv_files/requests.csv')

#Extracts a list of all unique users in dataframe
users = calc.extract_users(data)

#Create a datachunk for testing the API
df_chunks = calc.split_user_df(data, users[0]) 
df_chunks[0].to_csv('csv_files/datachunk.csv', index=False)

vect = calc.create_vectorizer(data)

In [ ]:
#32893 requests made by user-1003
data.loc[data['userID'] == ' user-1003']

In [ ]:
#32714 requests made by user-1004
data.loc[data['userID'] == ' user-1004']

In [ ]:
#68413 requests made by user-1005
data.loc[data['userID'] == ' user-1005']

In [3]:
"""
Code for creating csv-file containing information about average request frequency, average number of tokens per 5 mins, 
longest consecutive chain of repetitive requests for every chunk of 50 requests for each user
"""

df_calculations = pd.DataFrame()
for user in users:
    data_chunks = calc.split_user_df(data, user)
    avg_req_frequences = []
    avg_tokens = []
    repetitions = []
    var_scores = []
    sequence_time = []

    for i in range(0, len(data_chunks)):
        avg_req_frequences.append(calc.calc_avg_timediff(data_chunks[i]))
        avg_tokens.append(calc.avg_tokens_5mins(data_chunks[i]))
        repetitions.append(calc.longest_consec(data_chunks[i]))
        var_scores.append(calc.get_variance_score(data_chunks[i], vect))
        sequence_time.append(calc.sequence_time_length(data_chunks[i]))
    
    user_df = pd.DataFrame()
    user_df['request_freq'] = avg_req_frequences
    user_df['avg_tokens'] = avg_tokens
    user_df['longest_consec'] = repetitions
    user_df['var_score'] = var_scores
    user_df['sequence_time'] = sequence_time
    user_df['user'] = user
    df_calculations = pd.concat([df_calculations, user_df])

df_calculations.reset_index(inplace=True)
df_calculations = df_calculations.drop(columns='index')
df_calculations.to_csv('csv_files/calculations.csv', index=False) 

In [4]:
data_calc = pd.read_csv('csv_files/calculations.csv')
data_calc

,request_freq,avg_tokens,longest_consec,var_score,sequence_time,user
0,2.327595,4.000000,16.0,394.114286,183.880,user-1003
1,2.422278,2.000000,20.0,376.380952,191.360,user-1003
2,2.133443,4.000000,11.0,394.114286,168.542,user-1003
3,1.865949,2.000000,14.0,438.380952,147.410,user-1003
4,2.641722,3.000000,14.0,363.257143,208.696,user-1003
...,...,...,...,...,...,...
3795,598.277932,0.147727,2.0,31.238095,26324.229,user-147
3796,387.771702,0.096774,9.0,27.285714,18225.270,user-754
3797,642.863652,0.111111,5.0,24.352381,29571.728,user-754
3798,898.434058,0.121019,3.0,32.495238,46718.571,user-641


In [5]:
df_spam = extract_spammers_scrapers(data_calc)
split_train_test(extract_normal(data_calc, df_spam), df_spam)